In [1]:
# # Version	            Python version	   Compiler	    Build tools	  cuDNN	  CUDA
# # tensorflow_gpu-2.10.0	   3.7-3.10	       MSVC 2019	Bazel 5.1.1	   8.1	  11.2

# # %pip install nvidia-pyindex
# # %pip install tensorflow-gpu==2.10.0
# # %pip install torch
# # %pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
# # %pip install transformers
# # %pip install transformers[torch]
# # %pip install accelerate>=0.26.0
# %pip install matplotlib seaborn

In [2]:
# import torch
# print("PyTorch Version:", torch.__version__)
# print("CUDA Available:", torch.cuda.is_available())
# print("CUDA Version:", torch.version.cuda)

# x = torch.rand(5, 3)
# print(x)

1. Read the Dataset

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
import ast

# Load your dataset
df = pd.read_csv("datasets/bert_data.csv")  # Ensure your dataset is in the correct CSV format

# Safely convert stringified lists back to Python lists
df['attention_mask'] = df['attention_mask'].apply(ast.literal_eval)
small_df = df.sample(n=1000, random_state=42).reset_index(drop=True)


Phase 1: Spell Correction

In [4]:
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from torch.utils.data import Dataset, DataLoader
import torch

# 1. Split the data into training and validation sets (80% train, 20% validation)
train_df, val_df = train_test_split(small_df, test_size=0.2, random_state=42)

# 2. Initialize the tokenizer
tokenizer = BertTokenizer.from_pretrained("sagorsarker/bangla-bert-base")

# 3. Custom Dataset for BERT
class BertDataset(Dataset):
    def __init__(self, df, tokenizer, max_length=64):
        self.sentences = df['sentence'].tolist()
        self.labels = df['label'].tolist()  # Add label support
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.sentences)

    def __getitem__(self, idx):
        sentence = self.sentences[idx]
        label = self.labels[idx]  # Extract label
        
        encoding = self.tokenizer.encode_plus(       # Tokenize and encode the sentence
            sentence,
            add_special_tokens=True,      # Add [CLS] and [SEP] tokens
            max_length=self.max_length,   # Set maximum length
            padding='max_length',         # Pad sequences to max_length
            truncation=True,              # Truncate sequences if needed
            return_attention_mask=True,   # Return attention mask
            return_tensors='pt'           # Return PyTorch tensors
        )
        
        return {
            'input_ids': encoding['input_ids'].squeeze(0),           # Token IDs
            'attention_mask': encoding['attention_mask'].squeeze(0), # Attention mask
            'labels': torch.tensor(label, dtype=torch.long)          # Label as tensor
        }

# 4. Prepare datasets
train_dataset = BertDataset(train_df, tokenizer)
val_dataset = BertDataset(val_df, tokenizer)

# Example DataLoader
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=True)


c:\Users\thetr\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:

# # Inspect the batch
# for batch in train_loader:
#     for k, v in batch.items():
#         print(f"{k}: {v.shape}")  # Expected: [batch_size, seq_length]
#     break

In [6]:
# 5. Load pre-trained BERT model
model = BertForSequenceClassification.from_pretrained("sagorsarker/bangla-bert-base", num_labels=2)

# 6. Define training arguments
training_args = TrainingArguments(
    output_dir='./results',          # Output directory for model and logs
    num_train_epochs=3,              # Number of training epochs
    per_device_train_batch_size=8,   # Batch size for training
    per_device_eval_batch_size=8,    # Batch size for evaluation
    warmup_steps=500,                # Number of warmup steps
    weight_decay=0.01,               # Strength of weight decay
    logging_dir='./logs',            # Directory for logs
    logging_steps=10,                # Log every 10 steps
    evaluation_strategy="epoch",     # Evaluate every epoch
    save_strategy="epoch",           # Save the model after every epoch
    load_best_model_at_end=True      # Load the best model at the end
)

# 7. Initialize the Trainer
trainer = Trainer(
    model=model,                     # Model to train
    args=training_args,              # Training arguments
    train_dataset=train_dataset,     # Training dataset
    eval_dataset=val_dataset,        # Validation dataset
    tokenizer=tokenizer              # Tokenizer used to process input
)

trainer.train()

results = trainer.evaluate()
print(results)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at sagorsarker/bangla-bert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\thetr\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
C:\Users\thetr\AppData\Local\Temp\ipykernel_15356\2007751912.py:20: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
  3%|▎         | 10/300 [00:35<14:37,  3.03s/it]

{'loss': 0.6733, 'grad_norm': 7.260274410247803, 'learning_rate': 1.0000000000000002e-06, 'epoch': 0.1}


  7%|▋         | 20/300 [01:05<14:44,  3.16s/it]

{'loss': 0.6703, 'grad_norm': 5.957991600036621, 'learning_rate': 2.0000000000000003e-06, 'epoch': 0.2}


 10%|█         | 30/300 [01:37<13:55,  3.09s/it]

...

{'eval_loss': 0.13677270710468292, 'eval_runtime': 15.673, 'eval_samples_per_second': 12.761, 'eval_steps_per_second': 1.595, 'epoch': 3.0}


Saving vocabulary to ./results\checkpoint-300\vocab.txt: vocabulary indices are not consecutive. Please check that the vocabulary is not corrupted!
Saving vocabulary to ./results\checkpoint-300\vocab.txt: vocabulary indices are not consecutive. Please check that the vocabulary is not corrupted!
Saving vocabulary to ./results\checkpoint-300\vocab.txt: vocabulary indices are not consecutive. Please check that the vocabulary is not corrupted!
Saving vocabulary to ./results\checkpoint-300\vocab.txt: vocabulary indices are not consecutive. Please check that the vocabulary is not corrupted!
Saving vocabulary to ./results\checkpoint-300\vocab.txt: vocabulary indices are not consecutive. Please check that the vocabulary is not corrupted!
Saving vocabulary to ./results\checkpoint-300\vocab.txt: vocabulary indices are not consecutive. Please check that the vocabulary is not corrupted!
Saving vocabulary to ./results\checkpoint-300\vocab.txt: vocabulary indices are not consecutive. Please check th

{'train_runtime': 1944.4213, 'train_samples_per_second': 1.234, 'train_steps_per_second': 0.154, 'train_loss': 0.25302942625557384, 'epoch': 3.0}


100%|██████████| 25/25 [00:14<00:00,  1.74it/s]

{'eval_loss': 0.13203367590904236, 'eval_runtime': 14.7981, 'eval_samples_per_second': 13.515, 'eval_steps_per_second': 1.689, 'epoch': 3.0}


Evaluate Model Performance on Validation Set

In [7]:
# Evaluate the model on the validation set
results = trainer.evaluate()
print("Evaluation Results:", results)


100%|██████████| 25/25 [00:14<00:00,  1.75it/s]

Evaluation Results: {'eval_loss': 0.13203367590904236, 'eval_runtime': 14.9007, 'eval_samples_per_second': 13.422, 'eval_steps_per_second': 1.678, 'epoch': 3.0}


Make Predictions on New Data

In [8]:
# Sample new sentences for inference
new_sentences = [
    "গাজীপুরের কালিয়াকৈর উপজেলার তেলিরচালা এলাকায়",  # Correct sentence
    "গাজীপ৳ুরের ালিয়াকৈর উপজেলংার তেলিচরালা এলাকা়"  # Sentence with errors
]

# Preprocess and tokenize new sentences
inputs = tokenizer(new_sentences, return_tensors='pt', padding=True, truncation=True, max_length=128)

# Move inputs to model's device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
inputs = {key: val.to(device) for key, val in inputs.items()}

# Make predictions
model.eval()
with torch.no_grad():
    outputs = model(**inputs)
    predictions = torch.argmax(outputs.logits, axis=-1)

# Interpret predictions
for sentence, pred in zip(new_sentences, predictions):
    label = "Correct" if pred == 0 else "Incorrect"
    print(f"Sentence: {sentence}\nPrediction: {label}\n")


Sentence: গাজীপুরের কালিয়াকৈর উপজেলার তেলিরচালা এলাকায়
Prediction: Correct

Sentence: গাজীপ৳ুরের ালিয়াকৈর উপজেলংার তেলিচরালা এলাকা়
Prediction: Incorrect



Save the Trained Model

In [14]:
# Save the model and tokenizer
model.save_pretrained("./brammerly")
tokenizer.save_pretrained("./brammerly")
print("Model and tokenizer saved successfully!")


Model and tokenizer saved successfully!


Reload the model

In [15]:
from transformers import BertForSequenceClassification, BertTokenizer

model = BertForSequenceClassification.from_pretrained("./brammerly")
tokenizer = BertTokenizer.from_pretrained("./brammerly")


Inspect Misclassifications

In [16]:
# batch = {k: v.to(device) for k, v in batch.items()}

# # Debug shapes
# print("Input IDs shape:", batch['input_ids'].shape)
# print("Attention Mask shape:", batch['attention_mask'].shape)

# # Forward pass
# with torch.no_grad():
#     outputs = model(input_ids=batch['input_ids'], attention_mask=batch['attention_mask'])
#     logits = outputs.logits
#     preds = torch.argmax(logits, dim=-1)

# print("Predictions:", preds)


In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

# Create predictions for validation data
model.eval()
predictions = []
true_labels = []

for batch in val_dataset:  # val_dataloader is your DataLoader for validation data
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(input_ids=batch['input_ids'], attention_mask=batch['attention_mask'])
        logits = outputs.logits
        preds = torch.argmax(logits, dim=-1)

    predictions.extend(preds.cpu().numpy())
    true_labels.extend(batch['labels'].cpu().numpy())

# Convert to DataFrame for easy inspection
val_df['predicted_label'] = predictions
val_df['true_label'] = true_labels

# Inspect misclassified examples
misclassified = val_df[val_df['predicted_label'] != val_df['true_label']]
print("Misclassified Examples:")
print(misclassified[['sentence', 'true_label', 'predicted_label']])

Confusion Matrix

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix

# Generate confusion matrix
conf_matrix = confusion_matrix(true_labels, predictions)

# Plot confusion matrix
plt.figure(figsize=(6, 5))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", xticklabels=["Correct", "Incorrect"], yticklabels=["Correct", "Incorrect"])
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix")
plt.show()


Compute Accuracy, Precision, Recall, and F1 Score

In [ ]:
from sklearn.metrics import classification_report

# Generate a classification report
report = classification_report(true_labels, predictions, target_names=["Correct", "Incorrect"])
print("Classification Report:")
print(report)


Phase 2: Grammar Correction

In [ ]:
# Install GECToR dependencies (done once)
# !pip install allennlp allennlp-models

from allennlp.predictors import Predictor
from allennlp_models.pretrained import load_predictor

# Load GECToR pre-trained model
predictor = load_predictor("https://storage.googleapis.com/allennlp-public-models/roberta-base-2020.06.09.tar.gz")

# Example prediction
example_sentence = "তোমরা ভুলগুলো চিহ্নিত করনি।"
prediction = predictor.predict(sentence=example_sentence)

# Output
print("Original:", example_sentence)
print("Corrected:", prediction["tokens"])


GECToR Fine-Tuning for Bangla Grammar Correction

In [ ]:
# Clone GECToR Repo
git clone https://github.com/grammarly/gector.git
cd gector

# Install Dependencies
pip install -r requirements.txt

# Train GECToR
python train.py \
    --train_set "data/train.txt" \
    --dev_set "data/dev.txt" \
    --model_dir "output/bangla_model" \
    --pretrained_transformer "sagorsarker/bangla-bert-base" \
    --vocab_path "data/vocab" \
    --batch_size 32


Fine-Tune T5 for Sentence-Level Grammar Correction

In [ ]:
from transformers import T5ForConditionalGeneration, T5Tokenizer, Trainer, TrainingArguments
from datasets import Dataset

# Initialize tokenizer and model
tokenizer = T5Tokenizer.from_pretrained("t5-small")  # Replace with Bangla-supported T5
model = T5ForConditionalGeneration.from_pretrained("t5-small")

# Prepare data
train_data = {"input_text": df['synth_spelling_errors'].tolist(), 
              "target_text": df['cleaned_text'].tolist()}

train_dataset = Dataset.from_dict(train_data)

def preprocess_function(examples):
    inputs = ["correct: " + text for text in examples['input_text']]
    targets = examples['target_text']
    model_inputs = tokenizer(inputs, max_length=128, truncation=True, padding="max_length")
    labels = tokenizer(targets, max_length=128, truncation=True, padding="max_length")
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Tokenize data
tokenized_train = train_dataset.map(preprocess_function, batched=True)

# Training arguments
training_args = TrainingArguments(
    output_dir="./t5_bangla_results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train
)

# Start training
trainer.train()

# Save model
model.save_pretrained("./t5_bangla_finetuned")


Phase 3: Sentence Correction

In [ ]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

# Load T5 model and tokenizer
model_name = "google/mt5-small"  # Use T5 pre-trained model for multilingual tasks
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

# Tokenize data for T5
def preprocess_function(examples):
    inputs = [f"grammar: {i}" for i in examples["incorrect_sentence"]]
    outputs = examples["cleaned_text"]
    model_inputs = tokenizer(inputs, text_target=outputs, max_length=128, truncation=True)
    return model_inputs

# Tokenize train and validation sets
from datasets import Dataset
train_dataset = Dataset.from_pandas(train_data)
val_dataset = Dataset.from_pandas(val_data)

train_dataset = train_dataset.map(preprocess_function, batched=True)
val_dataset = val_dataset.map(preprocess_function, batched=True)

# Fine-tune the model
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer

training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    predict_with_generate=True,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

trainer.train()
trainer.save_model("t5_sentence_correction_model")


Integrate GECToR, T5, and BERT

In [ ]:
def correction_pipeline(sentence, gector_model, t5_model, bert_model, tokenizer):
    # Step 1: GECToR Token Correction
    gector_corrected_sentence = gector_model.correct(sentence)

    # Step 2: T5 Sentence-Level Correction
    input_text = "correct: " + gector_corrected_sentence
    inputs = tokenizer(input_text, return_tensors="pt")
    t5_outputs = t5_model.generate(**inputs)
    t5_corrected_sentence = tokenizer.decode(t5_outputs[0], skip_special_tokens=True)

    # Step 3: BERT Error Verification
    inputs = tokenizer(t5_corrected_sentence, return_tensors="pt", padding="max_length", truncation=True, max_length=128)
    outputs = bert_model(**inputs)
    logits = outputs.logits
    prediction = torch.argmax(logits, dim=-1)

    return {"original": sentence, 
            "gector": gector_corrected_sentence, 
            "t5": t5_corrected_sentence, 
            "bert_label": "Correct" if prediction.item() == 1 else "Incorrect"}

# Example usage
sentence = "গাজীপ৳ুরের ালিয়াকৈর উপজেলংার তেলিচরালা এলাকা়..."
result = correction_pipeline(sentence, gector_model, t5_model, bert_model, tokenizer)
print(result)
